In [14]:
!pip install torch
!pip install langchain
!pip install numpy
!pip install transformers
!pip install InstructorEmbedding
!pip install sentence_transformers
!pip install llama-cpp-python
!pip install faiss-gpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
import torch
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, TextStreamer, pipeline
from langchain import FAISS
from huggingface_hub import hf_hub_download
from langchain.llms import LlamaCpp
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name = "hkunlp/instructor-xl", model_kwargs = {"device": DEVICE}
)

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
loader = PyPDFDirectoryLoader("./data/")
docs = loader.load()
len(docs)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1024, chunk_overlap = 64) # play with these values later
texts = text_splitter.split_documents(docs)
len(texts)

In [ ]:
%%time
db = FAISS.from_documents(texts, embeddings)
db.save_local("./vectorstores/db_faiss")

In [4]:
model_path = hf_hub_download(
  repo_id="TheBloke/Llama-2-70B-chat-GGUF",
  filename="llama-2-70b-chat.Q4_K_M.gguf",
  resume_download=True,
  cache_dir="./models/",  #custom path for save the model
)

llama-2-70b-chat.Q4_K_M.gguf:   0%|          | 0.00/41.4G [00:00<?, ?B/s]

In [5]:
model = LlamaCpp(
        model_path = model_path,
        n_gpu_layers=100, #According to your GPU if you have
        n_batch=2048,
        verbose=True,
        f16_kv=True,
        n_ctx=4096,
        max_tokens=2048,
        callbacks=[StreamingStdOutCallbackHandler()]
    )

llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from ./models/models--TheBloke--Llama-2-70B-chat-GGUF/snapshots/6657b410331f892e2ea27eb435ef53aeefeedfd6/llama-2-70b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  8192, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  8192,  1024,     1,     1 ]
llama_model_loa

In [6]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful, honest and knowledgable health assistant. Always answer as highly as possible, trying to give medical diagnosis based on the symptoms provided by the patients. Your answers
should not include racist, sexist, harmful or unethical answers.
If a question does not make sense, or it is factually coherent, explain why instead of answering something incorrect. If you don't know answer to some question, please don't try to make up an answer, instead just say that
I don't know.
""".strip()

def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
  return f"""
  [INST] <<SYS>>
  {system_prompt}
  <</SYS>>

  {prompt} [/INST]
  """.strip()


In [7]:
SYSTEM_PROMPT = """
You are a helpful, respectful, honest and knowledgeable health assistant. Use the following pieces of information, the context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the questions. 
Always answer as highly as possible, trying to give medical diagnosis based on the symptoms provided by the patients. 
Your answers should not include racist, sexist, harmful or unethical answers.
If a question does not make sense, or it is factually coherent, explain why instead of answering something incorrect. 
If you don't know answer to some question, please don't try to make up an answer, instead just say that I don't know. Don't tell about yourself again and again.
Don't give the context (delimited by <ctx></ctx>) in the answer.
"""

template = generate_prompt(
    """
    ------
    <ctx>
    {context}
    </ctx>
    ------
    <hs>
    {history}
    </hs>
    ------

    Question: {question}
    """,
    system_prompt = SYSTEM_PROMPT
)

In [8]:
prompt = PromptTemplate(template = template, input_variables = ["history", "context", "question"])

In [9]:
db = FAISS.load_local('./vectorstores/db_faiss/', embeddings)

In [10]:
qa_chain = RetrievalQA.from_chain_type(
        llm = model,
        chain_type = "stuff",
        retriever = db.as_retriever(search_kwargs = {'k': 2}),
        return_source_documents = True, #explaining the answer
        chain_type_kwargs= {
            "verbose": True,
            "prompt": prompt,
            "memory": ConversationBufferMemory(
                memory_key="history",
                input_key="question"),
            
        },
    )


In [13]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/chat')
def chat_func():
    query = request.args.get('query')
    qa_chain(query)
    # print(res)
    # return res

@app.route('/awake')
def awake():
    return "hello"


@app.route('/clear_memory')
def clear_mem():
    global qa_chain
    # qa_chain.memory = ConversationBufferMemory(memory_key="history", input_key="question")
    qa_chain.combine_documents_chain.memory = ConversationBufferMemory(
                memory_key="history",
                input_key="question")
    return "Done"

app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.2.100:5000
Press CTRL+C to quit




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
  
You are a helpful, respectful, honest and knowledgeable health assistant. Use the following pieces of information, the context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the questions. 
Always answer as highly as possible, trying to give medical diagnosis based on the symptoms provided by the patients. 
Your answers should not include racist, sexist, harmful or unethical answers.
If a question does not make sense, or it is factually coherent, explain why instead of answering something incorrect. 
If you don't know answer to some question, please don't try to make up an answer, instead just say that I don't know. Don't tell about yourself again and again.
Don't give the context (delimited by <ctx></ctx>) in the answer.

  <</SYS>>

  
    ------
    <ctx>
    challenges to achieve.
■ Aβ2M AMYLOIDOSIS
Aβ2M amyloid is composed 

Llama.generate: prefix-match hit


  Hello Aryaman! I'm happy to assist you with any health-related questions or concerns you may have. Based on the information you provided, it seems like you are interested in learning more about Behçet syndrome and Aβ2M amyloididosis.

Behçet syndrome is a systemic vasculitis that can present with a wide range of symptoms, including skin and mucosal lesions, uveitis, arthritis, major arterial and venous vessel disease, and gastrointestinal and neurologic manifestations. It is relatively rare in the United States, but it is more common in countries such as Turkey, where it is most prevalent, with a prevalence of 1 in 250 adults.

Aβ2M amyloididosis, on the

10.0.0.221 - - [18/Nov/2023 07:27:03] "GET /awake HTTP/1.1" 200 -


 other hand, is a condition where abnormal proteins called Aβ2M amyloid accumulate in the body and cause damage to various organs and tissues. It can produce rheumatologic manifestations in patients undergoing long-term hemodialysis and, rarely, in patients with a hereditary form of the disease.

Given that you have mentioned these two conditions, I would like to ask you a few questions to better understand your concerns:

1. Have you or any family members been diagnosed with either Behçet syndrome or Aβ2M amyloididosis?
2. Are you experiencing any symptoms that may be related to these conditions, such as joint pain, skin rashes, or eye problems?
3. Are you concerned about your risk of developing either condition, given their relative rarity in the United States?

Please feel free to provide me with more information about your concerns or questions, and I'll do my best to assist you.


llama_print_timings:        load time =   72162.23 ms
llama_print_timings:      sample time =     223.80 ms /   407 runs   (    0.55 ms per token,  1818.59 tokens per second)
llama_print_timings: prompt eval time =   14767.60 ms /   133 tokens (  111.03 ms per token,     9.01 tokens per second)
llama_print_timings:        eval time =  213251.98 ms /   406 runs   (  525.25 ms per token,     1.90 tokens per second)
llama_print_timings:       total time =  229821.00 ms
[2023-11-18 07:29:01,319] ERROR in app: Exception on /chat [GET]
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/flask/app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/flask/app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/flask/app.py", line 867, in full


> Finished chain.

> Finished chain.
